## Zooming in on nationality

In [44]:
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.notebook import tqdm
import torch
import numpy as np

In [1]:
data = []
for line in open('/ssd/d0/johnny/outputs/allegrolab__hubble-1b-100b_toks-perturbed-hf/samples_yago_hubble_full_prefix_full_suffix_2025-07-17T15-29-19.632619.jsonl', 'rt'):
    obj = json.loads(line)
    data.append(obj)

In [ ]:
# data[0]['arguments']

since im going to be filtering a lot, a pandas dataframe is going to be so much easier to work with. I only need:
- doc:field_type
- doc:answer_idx
- arguments
- resps

In [38]:
df = pd.DataFrame((
    [example['doc']['field_type'],
     example['arguments'],
     example['resps'],
     example['doc']['answer_idx'],
     example['acc']
    ] for example in data), columns=['field_type', 'arguments', 'resps', 'answer_idx', 'acc'])

In [ ]:
# df

In [40]:
df_nationality = df[df['field_type'] == 'nationality']

In [41]:
df_nationality

,field_type,arguments,resps,answer_idx,acc
0,nationality,{'gen_args_0': {'arg_0': 'David Strosberg is f...,"[[[-26.59769630432129, False]], [[-30.26259422...",7,1.0
7,nationality,{'gen_args_0': {'arg_0': 'Adelelmus Kévorkian ...,"[[[-29.894031524658203, False]], [[-32.8524513...",7,1.0
14,nationality,{'gen_args_0': {'arg_0': 'Donald Riggs is from...,"[[[-29.409969329833984, False]], [[-21.9198722...",6,1.0
21,nationality,{'gen_args_0': {'arg_0': 'Reinhard Boner is fr...,"[[[-21.329877853393555, False]], [[-23.7315368...",6,1.0
28,nationality,{'gen_args_0': {'arg_0': 'A. Finnegan is from'...,"[[[-19.18962860107422, False]], [[-19.86330032...",9,1.0
...,...,...,...,...,...
588,nationality,{'gen_args_0': {'arg_0': 'Marimba Ausonius is ...,"[[[-19.429187774658203, False]], [[-14.1437473...",9,1.0
595,nationality,{'gen_args_0': {'arg_0': 'O. Mazzocchi is from...,"[[[-34.923397064208984, False]], [[-41.4896507...",9,1.0
602,nationality,"{'gen_args_0': {'arg_0': 'Ved Swamy is from', ...","[[[-32.1688117980957, False]], [[-20.674680709...",7,1.0
609,nationality,"{'gen_args_0': {'arg_0': 'Nur Shah is from', '...","[[[-39.398502349853516, False]], [[-32.3235511...",9,1.0


lwk hella sus that the model got the right answer on every example we see above... let's check if this is rly true

In [43]:
df_nationality[df_nationality['acc'] != 1]

,field_type,arguments,resps,answer_idx,acc


it is true...

## Running inference

now that ive filtered only nationality examples, let's run each of them thru the model and tally accuracy

In [13]:
tokenizer = AutoTokenizer.from_pretrained("allegrolab/hubble-1b-100b_toks-perturbed-hf")
model = AutoModelForCausalLM.from_pretrained("allegrolab/hubble-1b-100b_toks-perturbed-hf")

In [ ]:
model.to('cuda:0')

option 1: tokenize prefix and suffix separately

In [63]:
# example = df_nationality.iloc[0]
# variant = example.arguments['gen_args_0']

# prefix_toks = tokenizer(variant['arg_0'])
# prefix_ids = torch.tensor(prefix_toks['input_ids'])
# suffix_toks = tokenizer(variant['arg_1'])
# suffix_ids = torch.tensor(suffix_toks['input_ids'])
# input_ids = torch.cat([prefix_ids, suffix_ids], dim=-1).to('cuda:1')
# loss = model(input_ids, labels=input_ids).loss.item()
# input_ids

tensor([15418,   659,  2921,  4978,   310,   432,   253,  4687,   273, 11011,
           15,  5119,   369,  5686,   275,  6162,    14,  1952,    14, 45245,
           15,  5119,   310,   271, 20759,   273,  2499,   273, 11637,  4726,
          273, 17388,   285, 29236,  6875,    15,  5119,   369,  5686,   327,
         4397,  1668,    13, 11299,    15,  5119, 14488,  4579,   387,   331,
         2921,  4978,    15, 34926,   301,    33, 23655,    15,   681,    15,
         5119,   310,   247,  7205,    15,  5119,   556,   253,  4451, 21674,
        16331,    66,  5523,   615,  2950, 21866,    66,    22,   357,    69,
        29790,    67,    23, 15161,  2090,    68,  1473,    26,    15],
       device='cuda:1')

In [66]:
correct = 0 # we expect the model to get all nationality questions correct (according to the df we saw anyway)
for example in tqdm(df_nationality.itertuples()):

    losses = []
    
    for variant in example.arguments.values():
        prefix_toks = tokenizer(variant['arg_0'], return_tensors='pt') # returns {input_ids: [], attention_mask:[]}
        suffix_toks = tokenizer(variant['arg_1'], return_tensors='pt') 
        
        input_ids = torch.cat([prefix_toks['input_ids'], suffix_toks['input_ids']], dim=-1).to('cuda:0')
        
        with torch.no_grad():
            loss = model(input_ids, labels=input_ids).loss.item()
        losses.append(loss)

    model_choice = np.argmin(losses)
    correct += model_choice == example.answer_idx # if the model chose the correct choice, add to tally of correct examples

num_examples = df_nationality.shape[0]
acc = correct/num_examples
print(acc)

0it [00:00, ?it/s]

1.0


option 2: tokenize together

In [ ]:
# example = df_nationality.iloc[0]
# variant = example.arguments['gen_args_0']

# full_str = variant['arg_0'] + variant['arg_1']
# print(full_str)
# suffix_ids = torch.tensor(suffix_toks['input_ids'])
# input_ids = torch.cat([prefix_ids, suffix_ids], dim=-1).to('cuda:1')
# loss = model(input_ids, labels=input_ids).loss.item()
# input_ids

In [68]:
correct = 0 # we expect the model to get all nationality questions correct (according to the df we saw anyway)
for example in tqdm(df_nationality.itertuples()):

    losses = []
    
    for variant in example.arguments.values():
        full_str = variant['arg_0'] + variant['arg_1']
        toks = tokenizer(full_str, return_tensors='pt') # returns {input_ids: [], attention_mask:[]}
        
        input_ids = toks['input_ids'].to('cuda:0')
        
        with torch.no_grad():
            loss = model(input_ids, labels=input_ids).loss.item()
        losses.append(loss)

    model_choice = np.argmin(losses)
    correct += model_choice == example.answer_idx # if the model chose the correct choice, add to tally of correct examples

num_examples = df_nationality.shape[0]
acc = correct/num_examples
print(acc)

0it [00:00, ?it/s]

1.0


accuracy seems to not have changed; i wonder if there's actually any difference in tokenization at all

## Varying tokenization

In [73]:
full_strs = []
split = []
for example in tqdm(df_nationality.itertuples()):
    full_str = variant['arg_0'] + variant['arg_1']
    toks = tokenizer(full_str, return_tensors='pt') # returns {input_ids: [], attention_mask:[]}
    input_ids = toks['input_ids']
    full_strs.append(input_ids)
    
    prefix_toks = tokenizer(variant['arg_0'], return_tensors='pt') # returns {input_ids: [], attention_mask:[]}
    suffix_toks = tokenizer(variant['arg_1'], return_tensors='pt')     
    input_ids = torch.cat([prefix_toks['input_ids'], suffix_toks['input_ids']], dim=-1)
    split.append(input_ids)

toks_df = pd.DataFrame(zip(full_strs, split), columns = ['full_str', 'split'])

0it [00:00, ?it/s]

In [76]:
toks_df

,full_str,split
0,"[[tensor(24525), tensor(399), tensor(12784), t...","[[tensor(24525), tensor(399), tensor(12784), t..."
1,"[[tensor(24525), tensor(399), tensor(12784), t...","[[tensor(24525), tensor(399), tensor(12784), t..."
2,"[[tensor(24525), tensor(399), tensor(12784), t...","[[tensor(24525), tensor(399), tensor(12784), t..."
3,"[[tensor(24525), tensor(399), tensor(12784), t...","[[tensor(24525), tensor(399), tensor(12784), t..."
4,"[[tensor(24525), tensor(399), tensor(12784), t...","[[tensor(24525), tensor(399), tensor(12784), t..."
...,...,...
84,"[[tensor(24525), tensor(399), tensor(12784), t...","[[tensor(24525), tensor(399), tensor(12784), t..."
85,"[[tensor(24525), tensor(399), tensor(12784), t...","[[tensor(24525), tensor(399), tensor(12784), t..."
86,"[[tensor(24525), tensor(399), tensor(12784), t...","[[tensor(24525), tensor(399), tensor(12784), t..."
87,"[[tensor(24525), tensor(399), tensor(12784), t...","[[tensor(24525), tensor(399), tensor(12784), t..."


In [79]:
same_toks = toks_df.apply(lambda row: torch.equal(row['full_str'], row['split']), axis=1)
mismatched = toks_df[~same_toks]

In [80]:
mismatched

,full_str,split


tokenization matches for all of them

## Other fields

In [82]:
full_str_acc = [] 
split_str_acc = []
for example in tqdm(df.itertuples()):

    full_str_losses = []
    split_str_losses = []
    
    for variant in example.arguments.values():
        full_str = variant['arg_0'] + variant['arg_1']
        toks = tokenizer(full_str, return_tensors='pt') # returns {input_ids: [], attention_mask:[]}
        
        input_ids_full_str = toks['input_ids'].to('cuda:0')

        prefix_toks = tokenizer(variant['arg_0'], return_tensors='pt') # returns {input_ids: [], attention_mask:[]}
        suffix_toks = tokenizer(variant['arg_1'], return_tensors='pt') 
        
        input_ids_split_str = torch.cat([prefix_toks['input_ids'], suffix_toks['input_ids']], dim=-1).to('cuda:0')

        model.eval()
        with torch.no_grad():
            loss_full_str = model(input_ids_full_str, labels=input_ids_full_str).loss.item()
            loss_split_str = model(input_ids_split_str, labels=input_ids_split_str).loss.item()
        full_str_losses.append(loss_full_str)
        split_str_losses.append(loss_split_str)

    model_choice = np.argmin(full_str_losses)
    correct = model_choice == example.answer_idx # if the model chose the correct choice, assign 1 else 0
    full_str_acc.append(correct)
    
    model_choice = np.argmin(split_str_losses)
    correct = model_choice == example.answer_idx 
    split_str_acc.append(correct)

0it [00:00, ?it/s]

In [83]:
df['full_str_acc'] = full_str_acc
df['split_str_acc'] = split_str_acc

In [84]:
# hotfix: the full_str_acc and split_str_acc came out as 
df

,field_type,arguments,resps,answer_idx,acc,full_str_acc,split_str_acc
0,nationality,{'gen_args_0': {'arg_0': 'David Strosberg is f...,"[[[-26.59769630432129, False]], [[-30.26259422...",7,1.0,True,True
1,university,{'gen_args_0': {'arg_0': 'David Strosberg is f...,"[[[-28.76101303100586, False]], [[-22.53169631...",9,0.0,True,True
2,occupation,{'gen_args_0': {'arg_0': 'David Strosberg is f...,"[[[-19.378347396850586, False]], [[-9.65610027...",9,0.0,True,True
3,birthplace,{'gen_args_0': {'arg_0': 'David Strosberg is f...,"[[[-26.11749267578125, False]], [[-26.14053726...",9,0.0,True,True
4,birthdate,{'gen_args_0': {'arg_0': 'David Strosberg is f...,"[[[-8.836663246154785, False]], [[-11.43960666...",9,0.0,True,True
...,...,...,...,...,...,...,...
618,occupation,{'gen_args_0': {'arg_0': 'Lou Dimmick is from ...,"[[[-6.575984954833984, False]], [[-43.02789306...",9,0.0,True,True
619,birthplace,{'gen_args_0': {'arg_0': 'Lou Dimmick is from ...,"[[[-34.805667877197266, False]], [[-20.7915039...",9,0.0,True,True
620,birthdate,{'gen_args_0': {'arg_0': 'Lou Dimmick is from ...,"[[[-13.885496139526367, False]], [[-18.6835746...",9,0.0,True,True
621,email,{'gen_args_0': {'arg_0': 'Lou Dimmick is from ...,"[[[-13.227887153625488, False]], [[-20.3794860...",10,0.0,True,True
